In [ ]:
import cv2
import numpy as np
import time
from pynq_dpu import DpuOverlay

In [ ]:
IMAGE_PATH = "img/jinrikisha-911722.JPEG"
MODEL_PATH = "/home/ubuntu/Kria-PYNQ/movenet_kr260_vai25.xmodel"

In [ ]:
_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94

MEANS = [_B_MEAN,_G_MEAN,_R_MEAN]

def resize_shortest_edge(image, size):
    H, W = image.shape[:2]
    if H >= W:
        nW = size
        nH = int(float(H)/W * size)
    else:
        nH = size
        nW = int(float(W)/H * size)
    return cv2.resize(image,(nW,nH))

def mean_image_subtraction(image, means):
    B, G, R = cv2.split(image)
    B = B - means[0]
    G = G - means[1]
    R = R - means[2]
    image = cv2.merge([R, G, B])
    return image

def BGR2RGB(image):
    B, G, R = cv2.split(image)
    image = cv2.merge([R, G, B])
    return image

def central_crop(image, crop_height, crop_width):
    image_height = image.shape[0]
    image_width = image.shape[1]
    offset_height = (image_height - crop_height) // 2
    offset_width = (image_width - crop_width) // 2
    return image[offset_height:offset_height + crop_height, offset_width:
                 offset_width + crop_width, :]

def normalize(image):
    image=image/256.0
    image=image-0.5
    image=image*2
    return image

def preprocess_fn(image, crop_height = 192, crop_width = 192):
    image = resize_shortest_edge(image, 256)
    image = mean_image_subtraction(image, MEANS)
    image = central_crop(image, crop_height, crop_width)
    return image

In [ ]:
print(f"Loading DPU Overlay and Model: {MODEL_PATH}...")
overlay = DpuOverlay("dpu.bit")
overlay.load_model(MODEL_PATH)
dpu_runner = overlay.runner

In [ ]:
inputTensors = dpu_runner.get_input_tensors()
outputTensors = dpu_runner.get_output_tensors()

shapeIn = tuple(inputTensors[0].dims)
shapeOut = tuple(outputTensors[0].dims)

print(f" Model Loaded!")
print(f"   Input Tensor : {shapeIn} ({inputTensors[0].dtype})")
print(f"   Output Tensor: {shapeOut} ({outputTensors[0].dtype})")

In [ ]:
output_data = [np.empty(shapeOut, dtype=np.int8, order="C")]
input_data = [np.empty(shapeIn, dtype=np.int8, order="C")]
image = input_data[0]

In [ ]:
# ----------------------------------------
# 2. PRE-PROCESSING
# ----------------------------------------
print(f"Loading {IMAGE_PATH}...")
img = preprocess_fn(cv2.imread(IMAGE_PATH))
if img is None:
    print("Error: Could not load image. Check filename.")
    exit()

# Resize to what the model expects
input_height, input_width = shapeIn[1], shapeIn[2]
resized_img = cv2.resize(img, (input_width, input_height))

# Prepare the Input Buffer
# Vitis AI DPU expects data in the buffer we allocate
input_data = [np.empty(shapeIn, dtype=np.int8, order="C")]
output_data = [np.empty(shapeOut, dtype=np.int8, order="C")]

In [ ]:
# ----------------------------------------
# 3. INFERENCE (FPGA TIME!)
# ----------------------------------------
print("Running DPU Execution...")
start_t = time.time()

image[0,...] = img.reshape(shapeIn[1:])
job_id = dpu_runner.execute_async(input_data, output_data)
dpu_runner.wait(job_id)

end_t = time.time()
print(f"Inference Time: {(end_t - start_t)*1000:.2f} ms")